In [1]:
from tqdm import tqdm
import argparse
from utils import *
from freebase import *
from propagation import *
import random



parser = argparse.ArgumentParser()
parser.add_argument("--dataset", type=str,
                    default="webqsp", help="choose the dataset from {webqsp, cwq}.")
parser.add_argument("--max_length", type=int,
                    default=1024, help="the max length of LLMs output.")
parser.add_argument("--temperature", type=float,
                    default=0., help="the temperature")
parser.add_argument("--llm", type=str,
                    default="llama", help="choose base LLM model from {llama, gpt-3.5-turbo, gpt-4}.")
parser.add_argument("--openai_api_key", type=str,
                    default="", help="if the LLM is gpt-3.5-turbo or gpt-4, you need add your own openai api key.")
args = parser.parse_args("")


datas, question_string = prepare_dataset(args.dataset)

In [2]:
relations = []
for data in tqdm(datas):
    for i in data['Parses']:
        relation = i['InferentialChain']
        if relation is not None:
            relations += relation
relations = set(relations)

100%|██████████| 1639/1639 [00:00<00:00, 408634.86it/s]


In [3]:
sparql_all_relations = """
PREFIX ns: <http://rdf.freebase.com/ns/>
SELECT DISTINCT ?r
WHERE {
?e ?r ?e1 .
}
"""
r = execute_sparql(sparql_all_relations)
r = set([i['r']['value'].replace("http://rdf.freebase.com/ns/", "") for i in r if i['r']['value'].startswith("http://rdf.freebase.com/ns/")])

In [ ]:
filter_relations = list(r - relations)

In [5]:
set([i.split('.')[-1] for i in r]) - set([i.split('.')[-1]  for i in relations])

{'family_name_s',
 'episode_ended',
 'maximum_value',
 'house',
 'sub_division_of',
 'genbank_id',
 'solar_day',
 'associated_disorder',
 'final_issue_date',
 'became_organization',
 'floor_space_square_meters',
 'saf_region',
 'previous_episode',
 'argued',
 'medical_trials',
 'machines_simulated',
 'hall_of_fame',
 'length_meters',
 'ft_percentage',
 'width',
 'area_of_jurisdiction',
 'dedicated_to',
 'members_with_this_title',
 'vp_delegate_vote_tally',
 'manager',
 'convicts_transported',
 'to',
 'drives',
 'tattoo_name',
 'created',
 'watched_discussion_preferences',
 'acceptance_date',
 'artist',
 'callsign_meaning',
 'drug_formulations',
 'page_number',
 'link_color',
 'pipe_tobaccos_with_this_component',
 'provides_service',
 'next_intermediary',
 'average_rainfall_mm',
 'websites_owned',
 'survived_disasters',
 'businesses',
 'ballets_choreographed',
 'application_status',
 'locomotive',
 'super_class',
 'name_of_ballpark',
 'merged_with',
 'parent_coin_type',
 'contact_webpag

In [4]:
[i for i in r if 'alias' in i]

['user.druderman.default_domain.gene.alias',
 'base.natlang.property_alias.prop',
 'common.topic.alias',
 'base.natlang.property_alias.alias']